**Implementación de una Máquina de Estados Finito (FSM) Interactiva**

Este programa implementa un reconocedor de entradas de un usuario basado en una FSM predefinida. Requiere que el diseñador especifique la *tabla de transición*.

Primero, necesitamos instalar algunos paquetes:

In [11]:
%pip install spacy
!python -m spacy download es_core_news_sm


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Importamos algunas bibliotecas que necesitaremos:

In [12]:
import re
import es_core_news_sm
import spacy
from spacy import displacy
from spacy.matcher import Matcher


In [13]:
# Settings

SlotFillings = {
    "ORIGEN": {"valor": "", "needQuestion": True, "pregunta": "¿Desde dónde quieres salir?"},
    "DESTINO": {"valor": "", "needQuestion": True, "pregunta": "¿A dónde quieres ir?"},
    "FECHA": {"valor": "", "needQuestion": True, "pregunta": "¿Qué día quieres salir?"},
    "HORA": {"valor": "", "needQuestion": True, "pregunta": "¿A qué hora quieres salir?"},
    "IDA/VUELTA": {"valor": "", "needQuestion": True, "pregunta": "¿Quieres ida y vuelta?"},
    "FECHA_VUELTA": {"valor": "", "needQuestion": True, "pregunta": "¿Qué día quieres volver?"},
    "HORA_VUELTA": {"valor": "", "needQuestion": True, "pregunta": "¿A qué hora quieres volver?"}
}

_ERRORQUESTION = "Lo siento, no te he entendido. ¿Podrías repetirlo?"
_WELCOMEQUESTION = "Hola, ¿en qué puedo ayudarte?"
_LOGOQUESTION = """                                _____
                       __...---'-----`---...__
                  _===============================
 ______________,/'      `---..._______...---'
(____________LL). .    ,--'
 /    /.---'       `. /
'--------_  - - - - _/
          `~~~~~~~~'"""



# agrego patrones de reconocimiento de ORIGEN Y DESTINO
def matcherPattern(nlp):
    matcher = Matcher(nlp.vocab)
    pattern_origen = [{"LOWER": {"IN": ["de", "desde"]}}, {"ENT_TYPE": {"IN": ["GPE", "LOC"]}}]
    pattern_destino = [{"LOWER": {"IN": ["ir", "voy"]}, "OP": "?"}, {"LOWER": "a"}, {"ENT_TYPE": {"IN": ["GPE", "LOC"]}}]
    pattern_fecha = [{"LOWER": "el"}, {"LOWER": "dia", "OP": "?"}, {"IS_DIGIT": True}, {"LOWER": "de"}, {"IS_ALPHA": True}]
    pattern_hora = [{"LOWER": "a"}, {"LOWER": "las"}, {"IS_DIGIT": True}, {"TEXT": {"REGEX": "^(horas|:\d{2})?$"}}]
    pattern_solo_ida = [{"LOWER": "solo", "OP": "?"}, {"LOWER": "ida"}]
    pattern_ida_vuelta = [{"LOWER": "ida", "OP": "?"}, {"LOWER": "y", "OP": "?"}, {"LOWER": "vuelta"}]
    
    matcher.add("ORIGEN", [pattern_origen])
    matcher.add("DESTINO", [pattern_destino])
    matcher.add("FECHA", [pattern_fecha])
    matcher.add("HORA", [pattern_hora])
    matcher.add("SOLO_IDA", [pattern_solo_ida])
    matcher.add("IDA_VUELTA", [pattern_ida_vuelta])

    return matcher

nlp = spacy.load('es_core_news_sm')
matcher = matcherPattern(nlp)

In [14]:
class Inicio:
    @staticmethod
    def run():
        print(_WELCOMEQUESTION)
        print(_LOGOQUESTION)
        print("¿En qué puedo ayudar?")
        return LeerRespuesta.run()

class LeerRespuesta:
    @staticmethod
    def run():
        try:
            respuestaUsuario = input()
            #respuestaUsuario = unidecode(respuestaUsuario)
            ProcesarRespuesta.run(respuestaUsuario)
        except Exception as error:
            print(_ERRORQUESTION)
            print(error)
            #return LeerRespuesta.run()
        
class ProcesarRespuesta:
    
    @staticmethod
    def run(respuestaUsuario):
        doc = nlp(respuestaUsuario)
        #"proceso el texto en busca de coincidencias de sigma"
        match_data = ProcesarRespuesta.LeerMatcherDep(doc)
        #dibujo para entender las relaciones
        #ProcesarRespuesta.dibujarEntyToken(doc) 
        return VerificarSlotFilling.run(match_data)

    @staticmethod
    def LeerMatcherDep(doc):
        matches = matcher(doc)
        slots = {}
        match_data = []
        for match_id, start, end in matches:
            string_id = nlp.vocab.strings[match_id]  
            matched_span = doc[start:end]
            slots[string_id] = matched_span.text
            match_data.append({
            'string_id': string_id,
            'matched_text': matched_span.text,
            'start': start,
            'end': end
            }) 
            print(f"Matches {string_id}: {matched_span.text} (start={start}, end={end})")
        return match_data

    
    @staticmethod
    def dibujarEntyToken(doc):
        for token in doc:
            print(token.text, token.dep_, token.head.text, token.head.pos_, [child for child in token.children])

        for ent in doc.ents:
            print(ent.text, ent.label_)
            
        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})
        displacy.render(doc, style="ent", jupyter=True)

In [15]:
class VerificarSlotFilling:
    
    @staticmethod
    def run(match_data):
        for match in match_data:
            string_id = match['string_id']
            matched_text = match['matched_text']
            if string_id in ["ORIGEN", "DESTINO", "FECHA", "HORA", "FECHA_VUELTA", "HORA_VUELTA"]:
                SlotFillings[string_id]["valor"] = matched_text  # save the matched span text
                SlotFillings[string_id]["needQuestion"] = False 

            if string_id in ['SOLO_IDA', 'IDA_VUELTA']:
                SlotFillings['IDA/VUELTA']['valor'] = matched_text
                SlotFillings['IDA/VUELTA']['needQuestion'] = False

            if string_id == 'IDA_VUELTA':
                SlotFillings['FECHA_VUELTA']['needQuestion'] = True
                SlotFillings['HORA_VUELTA']['needQuestion'] = True
                
        return SistemaDeDialogo.run()

In [16]:
class SistemaDeDialogo:
    
    @staticmethod
    def run():
        remaining_questions = [key for key, value in SlotFillings.items() if value["needQuestion"] and key not in ["IDA_VUELTA", "FECHA_VUELTA", "HORA_VUELTA"]]
        if not remaining_questions:
            if SlotFillings["SOLO_IDA"]["needQuestion"] == False:
                print("Muchas gracias por preferirnos")
                print('&'.join([f"{key}={value['valor']}" for key, value in SlotFillings.items()]))
            return
        else:
            next_question = remaining_questions[0]
            print(SlotFillings[next_question]["pregunta"])
            return LeerRespuesta.run()


In [17]:
Inicio().run()

Hola, ¿en qué puedo ayudarte?
                                _____
                       __...---'-----`---...__
                  _===============================
 ______________,/'      `---..._______...---'
(____________LL). .    ,--'
 /    /.---'       `. /
'--------_  - - - - _/
          `~~~~~~~~'
¿En qué puedo ayudar?
Matches ORIGEN: desde santiago (start=3, end=5)
Matches DESTINO: a concepcion (start=5, end=7)
Matches FECHA: el dia 14 de marzo (start=8, end=13)
Matches HORA: a las 16 horas (start=13, end=17)
¿Quieres ida y vuelta?
Matches ORIGEN: desde santiago (start=3, end=5)
Matches DESTINO: a concepcion (start=5, end=7)
¿Quieres ida y vuelta?
Matches SOLO_IDA: solo ida (start=0, end=2)
Matches SOLO_IDA: ida (start=1, end=2)
Lo siento, no te he entendido. ¿Podrías repetirlo?
'SOLO_IDA'
